In [1]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

from concurrent.futures import ThreadPoolExecutor, wait

In [ ]:
# Парсим через pandas новости и рейтинги

def get_info(page=1):
    url = f'https://news.ycombinator.com/?p={page}'
    
    responce = requests.get(url)
    
    page = pd.read_html(responce.text)[2]
    page.drop([1], axis=1, inplace=True)
    page.rename(columns={0: 'index', 2: 'news'}, inplace=True)

    page = page[(~page['news'].isna()) & (page['news'] != 'More')]

    points = pd.DataFrame(
        page['news'][page['index'].isna()].reset_index(drop=True))
    points.rename(columns={'news':'points'}, inplace=True)

    points['contains_points'] = points['points'].str.contains("points")
    points['points'] = points['points'].str.split(' ')

    def set_points(row):
        if row['contains_points']:
            return row[0][0]
        else:
            return 0

    points['points'] = points.apply(set_points, axis=1)


    news = pd.DataFrame(page['news'][~page['index'].isna()].reset_index(drop=True))

    result = pd.concat([news, points['points']], axis=1)
    
    return result




news = pd.DataFrame(columns=['news', 'points'])
for i in range(9):
    news = pd.concat([news, get_info(i+1)])

---

In [ ]:
url_proger = 'https://tproger.ru/quiz/real-programmer'

In [ ]:
driver = webdriver.Chrome()
driver.get(url_proger)

driver.maximize_window()

scroll_to_bottom = "window.scrollTo(0, document.body.scrollHeight);"
driver.execute_script(scroll_to_bottom)

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.XPATH, '//*[@id="649c4c45d91e950aa12828ce"]'))
)
element.click()

answers = driver.find_elements(By.CLASS_NAME,'tp-answer__text')



#driver.close()
#driver.quit()

----

In [160]:
url_proger = 'https://tproger.ru/quiz/real-programmer'

df = pd.DataFrame(columns=('Answers', 'Percentage'))

driver = webdriver.Chrome()
driver.get(url_proger)

driver.maximize_window()

sleep(3)

driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.XPATH, '//*[@id="649c4c45d91e950aa12828ce"]'))
)
element.click()

#answers = driver.find_elements(By.CLASS_NAME, 'tp-answer__text')
#answers = [i.text for i in answers]

answers = driver.find_elements(By.CLASS_NAME, 'tp-answer__header')
#test = driver.find_elements(By.CLASS_NAME, 'tp-quiz-answer-list__item')

sleep(1)
temp = [i.text for i in answers]
temp_df = pd.DataFrame([i.split('\n') for i in temp], columns=('Answers', 'Percentage'))
temp_df


,Answers,Percentage
0,Hello world,91%
1,Сортировку «пузырьком»,3%
2,Для взлома аккаунта «ВКонтакте». Такая програм...,2%
3,Это вопрос с подвохом. Начинают они с починки ...,4%


---


In [28]:
url_proger = 'https://tproger.ru/quiz/real-programmer'

df = pd.DataFrame(columns=(0, 1, 2))

driver = webdriver.Chrome()
driver.get(url_proger)

driver.maximize_window()

sleep(3)

driver.execute_script(
    "window.scrollTo(0, window.scrollY + window.innerHeight);")

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.CLASS_NAME, 'tp-quiz-answer-list__item'))
)
element.click()
# r_answer = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true')
# f_answers = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')

sleep(1)
temp_r = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true').text
r_answer = temp_r.split('\n')
r_answer.append('correct')
r_answer_final = pd.DataFrame([r_answer])

#sleep(1)

if len(driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')) > 1:
    f_answers = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')
    f_answers = [i.text.split('\n') for i in f_answers]

    for i in f_answers:
        i.append('wrong answer')
    f_answers_final = pd.DataFrame(f_answers)
    
else:
    f_answers = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')
    f_answers = [i.text.split('\n') for i in f_answers]
    f_answers.append('wrong answer')
    f_answers_final = pd.DataFrame([f_answers])
        
result = pd.concat([r_answer_final, f_answers_final])
final_df = pd.concat([df, result])

final_df

,0,1,2
0,Hello world,91%,correct
0,Сортировку «пузырьком»,3%,wrong answer
1,Для взлома аккаунта «ВКонтакте». Такая програм...,2%,wrong answer
2,Это вопрос с подвохом. Начинают они с починки ...,4%,wrong answer


---

In [31]:
url_proger = 'https://tproger.ru/quiz/real-programmer'

df = pd.DataFrame(columns=('answer', '%', 'correctness'))
df.to_csv('./data/result_tproger',index=False)

driver = webdriver.Chrome()
driver.get(url_proger)

driver.maximize_window()

sleep(3)

driver.execute_script(
    "window.scrollTo(0, window.scrollY + window.innerHeight);")


while True:

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CLASS_NAME, 'tp-quiz-answer-list__item'))
    )
    element.click()

    sleep(1)

    temp_r = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true').text
    r_answer = temp_r.split('\n')
    r_answer.append('correct')
    r_answer_final = pd.DataFrame([r_answer])
    r_answer_final.rename(columns={0:'answer', 1:'%', 2:'correctness'}, inplace=True)

    if len(driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')) > 1:
        f_answers = driver.find_elements(
            By.CLASS_NAME, 'tp-answer--result-false')
        f_answers = [i.text.split('\n') for i in f_answers]

        for i in f_answers:
            i.append('wrong answer')
        f_answers_final = pd.DataFrame(f_answers)
        f_answers_final.rename(columns={0:'answer', 1:'%', 2:'correctness'}, inplace=True)

    else:
        temp_f = driver.find_elements(
            By.CLASS_NAME, 'tp-answer--result-false').text
        f_answers = temp_f.split('\n')
        f_answers.append('wrong answer')
        f_answers_final = pd.DataFrame([f_answers])
        f_answers_final.rename(columns={0:'answer', 1:'%', 2:'correctness'}, inplace=True)

    result = pd.concat([r_answer_final, f_answers_final])
    final_df = pd.read_csv('./data/result_tproger')
    final_df = pd.concat([final_df,result])
    final_df.to_csv('./data/result_tproger',index=False)

    button_next = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="quiz-app"]/div/div/div/div[3]/div[2]/button')))

    driver.execute_script("arguments[0].click();", button_next)
    sleep(3)



AttributeError: 'list' object has no attribute 'text'

---

In [38]:
url_proger = 'https://tproger.ru/quiz/real-programmer'

df = pd.DataFrame(columns=('answer', '%', 'correctness'))
df.to_csv('./data/result_tproger',index=False)

driver = webdriver.Chrome()
driver.get(url_proger)

driver.maximize_window()

sleep(3)

driver.execute_script(
    "window.scrollTo(0, window.scrollY + window.innerHeight);")


while True:

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CLASS_NAME, 'tp-quiz-answer-list__item'))
    )
    element.click()

    sleep(1)

    temp_r = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true').text
    r_answer = temp_r.split('\n')
    r_answer.append('correct')
    r_answer_final = pd.DataFrame([r_answer])
    r_answer_final.rename(columns={0:'answer', 1:'%', 2:'correctness'}, inplace=True)

    if len(driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')) > 1:
        f_answers = driver.find_elements(
            By.CLASS_NAME, 'tp-answer--result-false')
        f_answers = [i.text.split('\n') for i in f_answers]

        for i in f_answers:
            i.append('wrong answer')
        f_answers_final = pd.DataFrame(f_answers)
        f_answers_final.rename(columns={0:'answer', 1:'%', 2:'correctness'}, inplace=True)

    else:
        temp_f = driver.find_elements(
            By.CLASS_NAME, 'tp-answer--result-false')[0].text
        f_answers = temp_f.split('\n')
        f_answers.append('wrong answer')
        f_answers_final = pd.DataFrame([f_answers])
        f_answers_final.rename(columns={0:'answer', 1:'%', 2:'correctness'}, inplace=True)

    result = pd.concat([r_answer_final, f_answers_final])
    final_df = pd.read_csv('./data/result_tproger')
    final_df = pd.concat([final_df,result])
    final_df.to_csv('./data/result_tproger',index=False)

    button_next = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="quiz-app"]/div/div/div/div[3]/div[2]/button')))

    driver.execute_script("arguments[0].click();", button_next)
    sleep(3)



TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001031b2004 chromedriver + 4169732
1   chromedriver                        0x00000001031a9ff8 chromedriver + 4136952
2   chromedriver                        0x0000000102dff500 chromedriver + 292096
3   chromedriver                        0x0000000102e447a0 chromedriver + 575392
4   chromedriver                        0x0000000102e7f818 chromedriver + 817176
5   chromedriver                        0x0000000102e385e8 chromedriver + 525800
6   chromedriver                        0x0000000102e394b8 chromedriver + 529592
7   chromedriver                        0x0000000103178334 chromedriver + 3932980
8   chromedriver                        0x000000010317c970 chromedriver + 3950960
9   chromedriver                        0x0000000103160774 chromedriver + 3835764
10  chromedriver                        0x000000010317d478 chromedriver + 3953784
11  chromedriver                        0x0000000103152ab4 chro

In [39]:
final_df

,answer,%,correctness
0,Hello world,91%,correct
1,Сортировку «пузырьком»,3%,wrong answer
2,Для взлома аккаунта «ВКонтакте». Такая програм...,2%,wrong answer
3,Это вопрос с подвохом. Начинают они с починки ...,4%,wrong answer
4,"Нет, это фантастика",71%,correct
5,"Да, конечно",29%,wrong answer
6,"Буду искать ошибку в компиляторе, где-то она д...",73%,correct
7,Выключу комп и спокойно пойду спать — дело сде...,8%,wrong answer
8,Порадуюсь за себя и продолжу писать код,19%,wrong answer
9,"Изменятся, конечно",80%,correct


---

In [40]:
url_proger = 'https://tproger.ru/quiz/real-programmer'
result_file = './data/result_tproger.csv'

df = pd.DataFrame(columns=('answer', '%', 'correctness'))
df.to_csv(result_file, index=False)

driver = webdriver.Chrome()
driver.maximize_window()

driver.get(url_proger)

sleep(3)

driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")


while True:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'tp-quiz-answer-list__item'))
    )
    element.click()

    sleep(1)

    r_answer = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true').text.split('\n')
    r_answer.append('correct')
    r_answer_final = pd.DataFrame([r_answer], columns=['answer', '%', 'correctness'])

    if len(driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')) > 1:
        f_answers = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')
        f_answers = [i.text.split('\n') + ['wrong answer'] for i in f_answers]
        f_answers_final = pd.DataFrame(f_answers, columns=['answer', '%', 'correctness'])
    else:
        f_answer = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')[0].text.split('\n')
        f_answer.append('wrong answer')
        f_answers_final = pd.DataFrame([f_answer], columns=['answer', '%', 'correctness'])

    result = pd.concat([r_answer_final, f_answers_final])
    final_df = pd.read_csv(result_file)
    final_df = pd.concat([final_df, result])
    final_df.to_csv(result_file, index=False)

    button_next = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="quiz-app"]/div/div/div/div[3]/div[2]/button'))
    )
    driver.execute_script("arguments[0].click();", button_next)
    sleep(3)


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001025fe004 chromedriver + 4169732
1   chromedriver                        0x00000001025f5ff8 chromedriver + 4136952
2   chromedriver                        0x000000010224b500 chromedriver + 292096
3   chromedriver                        0x00000001022907a0 chromedriver + 575392
4   chromedriver                        0x00000001022cb818 chromedriver + 817176
5   chromedriver                        0x00000001022845e8 chromedriver + 525800
6   chromedriver                        0x00000001022854b8 chromedriver + 529592
7   chromedriver                        0x00000001025c4334 chromedriver + 3932980
8   chromedriver                        0x00000001025c8970 chromedriver + 3950960
9   chromedriver                        0x00000001025ac774 chromedriver + 3835764
10  chromedriver                        0x00000001025c9478 chromedriver + 3953784
11  chromedriver                        0x000000010259eab4 chro

In [41]:
final_df

,answer,%,correctness
0,Hello world,91%,correct
1,Сортировку «пузырьком»,3%,wrong answer
2,Для взлома аккаунта «ВКонтакте». Такая програм...,2%,wrong answer
3,Это вопрос с подвохом. Начинают они с починки ...,4%,wrong answer
4,"Нет, это фантастика",71%,correct
5,"Да, конечно",29%,wrong answer
6,"Буду искать ошибку в компиляторе, где-то она д...",73%,correct
7,Выключу комп и спокойно пойду спать — дело сде...,8%,wrong answer
8,Порадуюсь за себя и продолжу писать код,19%,wrong answer
9,"Изменятся, конечно",80%,correct


----

In [42]:
url_proger = 'https://tproger.ru/quiz/real-programmer'
result_file = './data/result_tproger.csv'

df = pd.DataFrame(columns=('answer', '%', 'correctness'))
df.to_csv(result_file, index=False)

driver = webdriver.Chrome()
driver.maximize_window()

driver.get(url_proger)

sleep(3)

driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")


while True:
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'tp-quiz-answer-list__item'))
        )
        element.click()

        sleep(1)

        r_answer = driver.find_element(By.CLASS_NAME, 'tp-answer--result-true').text.split('\n')
        r_answer.append('correct')
        r_answer_final = pd.DataFrame([r_answer], columns=['answer', '%', 'correctness'])

        if len(driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')) > 1:
            f_answers = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')
            f_answers = [i.text.split('\n') + ['wrong answer'] for i in f_answers]
            f_answers_final = pd.DataFrame(f_answers, columns=['answer', '%', 'correctness'])
        else:
            f_answer = driver.find_elements(By.CLASS_NAME, 'tp-answer--result-false')[0].text.split('\n')
            f_answer.append('wrong answer')
            f_answers_final = pd.DataFrame([f_answer], columns=['answer', '%', 'correctness'])

        result = pd.concat([r_answer_final, f_answers_final])
        final_df = pd.read_csv(result_file)
        final_df = pd.concat([final_df, result])
        final_df.to_csv(result_file, index=False)

        button_next = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="quiz-app"]/div/div/div/div[3]/div[2]/button'))
        )
        driver.execute_script("arguments[0].click();", button_next)
        sleep(3)
    except:
        break
    
final_df


,answer,%,correctness
0,Hello world,91%,correct
1,Сортировку «пузырьком»,3%,wrong answer
2,Для взлома аккаунта «ВКонтакте». Такая програм...,2%,wrong answer
3,Это вопрос с подвохом. Начинают они с починки ...,4%,wrong answer
4,"Нет, это фантастика",71%,correct
5,"Да, конечно",29%,wrong answer
6,"Буду искать ошибку в компиляторе, где-то она д...",73%,correct
7,Выключу комп и спокойно пойду спать — дело сде...,8%,wrong answer
8,Порадуюсь за себя и продолжу писать код,19%,wrong answer
9,"Изменятся, конечно",80%,correct
